# Chapter 1 - Differentiation Matrices

In [ ]:
using Plots
gr()

## Program 1

In [ ]:
function diffmaterr(N, fun, dfun, xlims=(-π, π))
    L = xlims[2] - xlims[1]
    
    h = L / N
    x = xlims[1] + (1:N)*h
    
    u = fun.(x)
    du = dfun.(x)
    
    e = ones(N)
    
    D1 = sparse(1:N, [2:N; 1], 2*e/3, N, N) - 
         sparse(1:N, [3:N; 1; 2], e/12, N, N)
    D = (D1 - D1') / h
    
    err = norm(D*u - du, Inf)
    
    return err
end


    

In [ ]:
fun(x) = exp(sin(x))
dfun(x) = cos(x) * fun(x)

In [ ]:
Nvec = 2 .^(3:12)

In [ ]:
err = diffmaterr.(Nvec, fun, dfun)

In [ ]:
scatter(Nvec, err, yaxis=:log10, xaxis=:log10, marker=5, label="Error")
plot!(Nvec, (1.0*Nvec).^(-4), label="N^(-4)")


In [ ]:
using Polynomials

In [ ]:
function lagrangei(i, N)
    
    den = 1
    xn = zeros(Rational{Int}, N-1)
    x = 0:(N-1)
    j = 1
    for k = 1:(i-1)
        xn[j] = x[k]  
        den = den * (x[i] - x[k])
        j += 1
    end
    for k = (i+1):N
        xn[j] = x[k]
        den = den * (x[i] - x[k])
        j += 1
    end
    
    p = poly(xn)
    return p/den
end


function dercoefs(N)
    
   
    pd = [polyder(lagrangei(i,N)) for i = 1:N]
    D = zeros(Rational{Int}, N,N)
    x = 0:(N-1)
    for j = 1:N
        for i = 1:N
            D[j,i] = polyval(pd[i], x[j])
        end
    end
    return D 
end    



In [ ]:
lagrangei(1,5)

In [ ]:
dercoefs(7)[4,:]

In [ ]:
function diffmat(N, P, h=1.0)#, xlims=(-π, π))
    nsten = P÷2
    icen = nsten+1
    Dloc = Float64.(dercoefs(P)[icen,:])
    o = ones(N)/h
    D = sparse(1:N, 1:N, ones(N)*Dloc[icen], N, N)

    for i = 1:nsten
        J1 = [(i+1):N; 1:i]
        D = D + sparse(1:N, J1, o*Dloc[icen+i], N, N)
        J2 = [(N-i+1):N; 1:(N-i)]
        D = D + sparse(1:N, J2, o*Dloc[icen-i], N, N)
    end
    return D
end

In [ ]:
diffmat(10, 3, 1.0)

In [ ]:
function diffmaterr2(x, D, fun, dfun)
    N = size(x,1)
        
    u = fun.(x)
    du = dfun.(x)
    
    err = norm(D*u - du, Inf)
    
    return err
end


In [ ]:
Nvec2 = 2 .^(4:16)
n = length(Nvec2)
err2 = zeros(n)

xmin = -π
xmax = π
L = xmax - xmin
    

P = 5
for i = 1:n
    N = Nvec2[i]
    h = L / N
    x = xmin + (1:N)*h
    D = diffmat(N, P, h)
    err2[i] = diffmaterr2(x, D, fun, dfun)
end

scatter(Nvec2, err2, yaxis=:log10, xaxis=:log10, marker=5, label="Error")
plot!(Nvec2, (1.0*Nvec2).^(-(P-1)), label=string(P-1))
